In [24]:
import os
import json

import pandas as pd
import numpy as np
from collections import Counter

from scipy.cluster.hierarchy import dendrogram, linkage
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.colors as mcolors

config = json.load(open('config.json'))

from openai import OpenAI

client = OpenAI(
    api_key=config['openai']['api_key']
)

category = {
    "Floral": ["Floral", "Black tea"],
    "Fruity": ["Berry", "Dried Fruit", "Other Fruit", "Citrus Fruit"],
    "Sour / Fermented": ["Sour", "Alcohol / Fermented"],
    "Vegetal": ["Olive Oil", "Raw", "Green / Vegetative", "Beany"],
    "Other": ["Papery / Musty", "Chemical"],
    "Roasted": ["Cereal", "Burnt", "Tobacco"],
    "Spices": ["Pungent", "Pepper", "Brown Spice"],
    "Nutty / Cocao": ["Nutty", "Cocao"],
    "Sweet": ["Brown Sugar", "Vanilla", "Sweet"]
}

category_colors = {
    "Floral": "#7F55B1",
    "Fruity": "#9D3434",
    "Sour / Fermented": "#FFC107",
    "Vegetal": "#6F8F3C",
    "Other": "gray",
    "Roasted": "#4E3C21",
    "Spices": "#EEE2B5",
    "Nutty / Cocao": "#B99470",
    "Sweet": "#9EC6F3"
}

roast_order = ['Light', 'Medium-Light', 'Medium', 'Medium-Dark', 'Dark', "Very Dark"]

outpath_base = './data/coffeereviewcom-over-7000-ratings-and-reviews/parsed_reviews'

In [25]:
data = pd.read_csv("./data/coffeereviewcom-over-7000-ratings-and-reviews/reviews_feb_2023.csv")
data = data[~(data['roast_level'].isna())]
data = data[~((data['bottom_line'].isna()) & (data['blind_assessment'].isna()))]
data['notes'] = data['notes'].astype(str)
data['roast_level'] = data['roast_level'].astype(str)
data = data.fillna('')
for col in ['rating', 'acidity_structure', 'aftertaste', 'aroma', 'body', 'flavor']:
    data[col] = data[col].astype(str)
    data[col] = data[col].replace('', np.nan).replace('NR', np.nan)  # Replace empty strings and 'NR' with NaN
    data[col] = data[col].astype(float)  # Convert to float, NaNs will be handled automatically

data.head()

,title,rating,acidity_structure,aftertaste,aroma,body,flavor,with_milk,agtron,blind_assessment,bottom_line,coffee_origin,est_price,notes,review_date,roast_level,roaster,roaster_location,url
0,Bolivia Manantial Gesha,93.0,9.0,8.0,9.0,8.0,9.0,,60/78,"Richly aromatic, floral-toned. Magnolia, cocoa...",This washed Boliva Gesha has all the aromatics...,"Caranavi, Bolivia",$30.00/12 ounces,"Produced by Angel Mamani Chambi, entirely of t...",January 2023,Medium-Light,Red Rooster Coffee Roaster,"Floyd, Virginia",https://www.coffeereview.com/review/bolivia-ma...
1,Yellow Pacamara Carbonic Maceration Nanolot,92.0,8.0,8.0,9.0,8.0,9.0,,60/78,"Crisply sweet-savory. White peach, hop flowers...",A carbonic-macerated Yellow Pacamara grown in ...,"Matagalpa growing region, Nicaragua",$160.00/12 ounces,Produced by Benjamin Weiner at Finca Idealista...,December 2022,Medium-Light,Eccentricity Coffee Co.,"Cleveland, Ohio",https://www.coffeereview.com/review/brix-break...
2,Ethiopia Gera Genji Challa,94.0,9.0,8.0,9.0,9.0,9.0,,59/77,"Delicately aromatic, complex. Lilac, cocoa nib...","A nuanced, multilayered washed Ethiopia cup wi...","Agaro Gera, Jimma Zone, Oromia State, Ethiopia",$28.00/12 ounces,Ethiopia coffees like this one are largely pro...,December 2022,Medium-Light,Mostra Coffee,"San Diego, California",https://www.coffeereview.com/review/ethiopia-g...
3,Yirgacheffe Mengesha Natural,94.0,9.0,8.0,9.0,9.0,9.0,,60/77,"High-toned, fruit-driven. Boysenberry, pear, c...",A fruit medley in a cup — think boysenberry an...,"Yirgacheffe growing region, southern Ethiopia",$20.50/12 ounces,Produced at Mengesha Farm from selections of i...,November 2022,Medium-Light,Regent Coffee,"Glendale, California",https://www.coffeereview.com/review/yirgacheff...
4,Tropical Summer Colombia La Sierra,93.0,9.0,8.0,9.0,8.0,9.0,,60/77,"Fruit-driven, crisply chocolaty. Goji berry, d...","An experimentally processed Colombia, sweetly ...","La Sierra, Cauca Department, Colombia",$18.99/8 ounces,Produced by smallholding farmers from trees of...,November 2022,Medium-Light,Merge Coffee Company,"Harrisonburg, Virginia",https://www.coffeereview.com/review/tropical-s...


## process review notes with OpenAI

In [ ]:
prompt = '''
Parse this coffee review to get the flavor notes and coffee processing method. For flavor notes, also classify it into one of the following categories (sub-categories):
- "Floral" ("Floral", "Black tea")
- "Fruity" ("Berry", "Dried Fruit"
- "Other Fruit", "Citrus Fruit")
- "Sour / Fermented" ("Sour", "Alcohol / Fermented")
- "Vegetal" ("Olive Oil", "Raw", "Green / Vegetative", "Beany")
- "Other" ("Papery / Musty", "Chemical")
- "Roasted" ("Cereal", "Burnt", "Tobacco")
- "Spices" ("Pungent", "Pepper", "Brown Spice")
- "Nutty / Cocao" ("Nutty", "Cocao")
- "Sweet" ("Brown Sugar", "Vanilla", "Sweet").

For coffee processing method, it must be either: "Washed", "Natural", "Honey", "Pulped Natural", "Anaerobic", "Wet Hulled", "Anaerobic Honey", "Carbonic Maceration", or "Other".

If the review does not mention any flavor notes, return an empty list. If the review does not mention any coffee processing method, return "Other". Return the results in a JSON format with the following structure: {"flavors": [{"flavor": "flavor note", "category": "category of flavor", "sub-category": "sub category flavor"}], "process": "coffee processing method"}.
'''

for i in data.index:
    outpath = f"{outpath_base}/review_{i}.json"
    
    if os.path.exists(outpath):
        print(f"Skipping review {i} as it has already been processed.")
        continue
    
    input_text = f"{data.at[i, 'blind_assessment']} {data.at[i, 'bottom_line']} {data.at[i, 'notes']}"
    
    try:
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": input_text}
            ],
            response_format={"type": "json_object"}
        )
        
        out = json.loads(response.choices[0].message.content)
        
        with open(outpath, 'w') as f:
            json.dump(out, f, indent=2)
        print(f"Processed review {i} and saved to {outpath}")
            
    except Exception as e:
        print(f"Error processing review {i}: {e}")


In [51]:
# output table of flavor notes for double checking
notes = []

for fpath in os.listdir(outpath_base):
    if fpath.endswith('.json'):
        with open(os.path.join(outpath_base, fpath), 'r') as f:
            review_data = json.load(f)
            idx = int(os.path.splitext(fpath)[0].split('_')[-1])
            
            for flavor in review_data.get("flavors", []):
                notes.append({
                    'Review ID': idx,
                    'Roast Level': data.at[idx, 'roast_level'],
                    "Review Text Snippet": flavor.get("flavor", ""),
                    "Category": flavor.get("category", ""),
                    "Sub-category": flavor.get("sub-category", ""),
                    'Url': data.at[idx, 'url'],
                })
notes = pd.DataFrame(notes)
notes.to_csv('./data/coffeereviewcom-over-7000-ratings-and-reviews/parsed_flavor_notes.csv', index=False)

# load the parsed reviews

In [26]:
rename = {
    'Nutty / Cocacao': 'Nutty / Cocao'
}
blacklist = [
    None,
    "",
    "None"
]
results = {}
categories = set()
sub_categories = set()
processing_methods = {}
for fpath in os.listdir(outpath_base):
    if fpath.endswith('.json'):
        with open(os.path.join(outpath_base, fpath), 'r') as f:
            review_data = json.load(f)
            idx = int(os.path.splitext(fpath)[0].split('_')[-1])
            results[idx] = review_data
            
            for flavor in review_data.get("flavors", []):
                cat_i = flavor.get("category", "")
                cat_i = rename.get(cat_i, cat_i)
                categories.add(cat_i)
                sub_categories.add(flavor["sub-category"])

categories = list(categories - {"", None})
sub_categories = list(sub_categories - {"", None})

indices = list(set(results.keys()) & set(data.index))

data_categories = pd.DataFrame(columns=categories, index=indices)
data_sub_categories = pd.DataFrame(columns=sub_categories, index=indices)
# fill with 0
data_categories.fillna(0, inplace=True)
data_sub_categories.fillna(0, inplace=True)

for idx, review_data in results.items():
    if idx not in indices:
        continue

    for flavor_i in review_data.get("flavors", []):
        category_i = flavor_i["category"]
        category_i = rename.get(category_i, category_i)
        data_categories.at[idx, category_i] = 1

        sub_category = flavor_i["sub-category"]
        if sub_category in blacklist:
            continue
        # data_sub_categories.at[idx, sub_category] += 1
        data_sub_categories.at[idx, sub_category] = 1
        
data = data.loc[data_categories.index, :]


/var/folders/sr/6n1kflr12vg9jkny1009t40m0000gn/T/ipykernel_33948/3937419476.py:34: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/sr/6n1kflr12vg9jkny1009t40m0000gn/T/ipykernel_33948/3937419476.py:35: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [28]:
# plot a heatmap of the percentage of each flavor note in each category
col = 'roast_level'

tmp = data_sub_categories.copy()
tmp[col] = data[col]
tmp_count = tmp.groupby(col).sum()
tmp = tmp_count.copy()
tmp_count = tmp_count.sum(axis=0)
# tmp = tmp.groupby('roast').sum()
for i in tmp.index:
    for flavor in tmp.columns:
        if tmp.at[i, flavor] == 0:
            continue
        tmp.at[i, flavor] = tmp.at[i, flavor] / len(data[data[col] == i])
        # tmp.at[i, flavor] = tmp.at[i, flavor] / tmp[data[col] == i][flavor].mean()

# re oder rows by roast order
if col in ['roast', 'roast_level']:
    tmp = tmp.reindex(roast_order)

threshold = 5
counts = Counter(data[col])
for i in tmp.index:
    if counts[i] < threshold:
        tmp.drop(i, inplace=True)
    
tmp

/var/folders/sr/6n1kflr12vg9jkny1009t40m0000gn/T/ipykernel_33948/1858339155.py:14: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.42358078602620086' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/sr/6n1kflr12vg9jkny1009t40m0000gn/T/ipykernel_33948/1858339155.py:14: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.043668122270742356' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/sr/6n1kflr12vg9jkny1009t40m0000gn/T/ipykernel_33948/1858339155.py:14: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.017467248908296942' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/sr/6n1kflr12vg9jkny

,Burnt,Vanilla,Chemical,Alcohol / Fermented,Raw / Green / Vegetative,Sour,Green / Vegetative,Raw,Papery / Musty,Brown Spice,...,Roasted,Black tea,Nutty,Fermented,Dried Fruit,Earth-toned,Tobacco,Other Fruit,Beany,Cocao
roast_level,,,,,,,,,,,,,,,,,,,,,
Light,0.004098,0.051913,0.002732,0.140710,0.000000,0.303279,0.040984,0.016393,0.012295,0.210383,...,0.000000,0.024590,0.375683,0.000000,0.116120,0.000000,0.027322,0.674863,0.000000,0.678962
Medium-Light,0.003334,0.041681,0.000667,0.089030,0.000000,0.248416,0.043014,0.014338,0.012671,0.226409,...,0.000333,0.010003,0.378793,0.001667,0.130377,0.000333,0.038680,0.655885,0.000000,0.739913
Medium,0.071752,0.055591,0.001939,0.108597,0.000000,0.207498,0.040724,0.028442,0.043310,0.148675,...,0.000000,0.005818,0.306399,0.005171,0.153846,0.000000,0.042663,0.552683,0.000000,0.785391
Medium-Dark,0.196615,0.074219,0.006510,0.101562,0.000000,0.157552,0.033854,0.031250,0.082031,0.183594,...,0.001302,0.001302,0.248698,0.003906,0.154948,0.000000,0.027344,0.473958,0.000000,0.760417
Dark,0.423581,0.043668,0.017467,0.074236,0.004367,0.104803,0.030568,0.043668,0.091703,0.170306,...,0.000000,0.004367,0.139738,0.004367,0.170306,0.000000,0.039301,0.423581,0.000000,0.663755
Very Dark,0.375648,0.033679,0.012953,0.054404,0.000000,0.158031,0.041451,0.054404,0.108808,0.116580,...,0.002591,0.010363,0.191710,0.000000,0.163212,0.000000,0.028497,0.378238,0.002591,0.660622


In [ ]:
# use make_subplots to plot a heatmap of the percentage of each flavor note in each category
# the number of columns is the number of sub-categories
# color each heatmap by the category color

count_max = tmp_count.max() * 1.05

col_widths = []
for category_i in category:
    favors = category[category_i]
    col_widths.append(len(set(favors) & set(tmp.columns)))
col_widths = np.array(col_widths)
col_widths = col_widths / col_widths.sum()
col_widths = np.append(col_widths, 3 / (len(col_widths) + 2))
col_widths = np.append(col_widths, 1 / (len(col_widths) + 2))
col_widths = col_widths / col_widths.sum()  # normalize to sum to 1


fig = make_subplots(
    cols=2,
    rows=len(category) + 2,
    subplot_titles=None,
    # subplot_titles=list(category.keys()),
    row_heights=col_widths.tolist(),
    column_widths=[0.9, 0.1],
    vertical_spacing=0.02,
    horizontal_spacing=0.01
)

for i, (category_i, favors) in enumerate(category.items()):
    color_i = category_colors.get(category_i, '#fff')
    flavors_i = list(set(favors) & set(tmp.columns))
    tmp_i = tmp[flavors_i]
    
    cmap = mcolors.LinearSegmentedColormap.from_list(
        f"cmap_{category_i}",
        ['white', color_i],
        N=120
    )
    cmap = [[0, 'white'], [1, color_i]]
    
    fig.add_trace(
        go.Heatmap(
            z=tmp_i.values.T,
            y=tmp_i.columns,
            x=tmp_i.index,
            colorscale=cmap,
            showscale=False,
            # name=category_i
        ),
        row=i+1, col=1
    )
    # hide the x-axis and y-axis labels
    fig.update_xaxes(
        # title_text=category_i,
        row=i+1, col=1,
        showticklabels=False,
    )
    fig.update_yaxes(
        # title_text="Categories" if i == 0 else "",
        row=i+1, col=1,
        showticklabels=True,
        ticks="outside",
    )
    
    # horizontal bar chart using tmp_count data
    fig.add_trace(
        go.Bar(
            x=tmp_count[flavors_i].values,
            y=flavors_i,
            orientation='h',
            showlegend=False,
            marker=dict(color=color_i, line=dict(width=0)),
        ),
        row=i+1, col=2
    )
    # hide x and y-axis labels
    fig.update_xaxes(
        row=i+1, col=2,
        showticklabels=True if i == 0 else False,
        range=[0, count_max],
        showgrid=False,
        # show axis labels on opposite side
        side='top' if i == 0 else 'bottom',
        # decrease font size of x-axis labels
        tickfont=dict(size=14),
        # show ticks outside
        ticks="outside" if i == 0 else None,
        # rotate the x-axis labels
        tickangle=-45 if i == 0 else 0
    )
    fig.update_yaxes(
        row=i+1, col=2,
        showticklabels=False,
        showgrid=False,
    )

# add bar chart of the number of reviews in each category
tmp_counts = data.value_counts('roast_level').reindex(roast_order)
fig.add_trace(
    go.Bar(
        x=tmp_counts.index,
        y=tmp_counts.values,
        showlegend=False,
        marker_color='#412F14',
    ),
    row=len(category) + 2, col=1
)
fig.update_xaxes(showticklabels=False, row=len(category) + 2, col=1)
fig.update_xaxes(showticklabels=False, row=len(category) + 1, col=1)


cmap = mcolors.LinearSegmentedColormap.from_list(
    "cmap_bar",
    ["#412F14", "#fbb309"],
    N=120
)
# convert each value in tmp_rating.values to between 0 and 1, where min is 60 and max is 100
tmp_rating = data.groupby('roast_level')[['rating']].mean().reindex(roast_order)
tmp_rating = tmp_rating.values.T[0]
tmp_rating = (tmp_rating - 85) / (100 - 85)
colors = [mcolors.to_hex(cmap(i)) for i in tmp_rating]

for i, (roast_i, color_i) in enumerate(zip(roast_order, colors)):
    tmp_i = data[data['roast_level'] == roast_i]
    fig.add_trace(
        go.Box(
            x=tmp_i['roast_level'],
            y=tmp_i['rating'],
            showlegend=False,
            marker=dict(
                color=color_i,
                line=dict(width=0)
            ),
        ),
        row=len(category) + 1, col=1
    )
    
fig.update_yaxes(
    row=len(category) + 1, col=1,
    showticklabels=True,
    range=[75,100]
)


fig.update_layout(
    height=600,
    width=1200,
    showlegend=False,
    paper_bgcolor="#e0d7cd",
    plot_bgcolor="#eae5df",
    font=dict(family="Helvetica, sans-serif", size=24, color="#000"),
)

# fig.show(renderer='png', height=1500, width=800)
fig.write_image('./plots/kaggle_coffee_flavor_notes.svg', height=2200, width=1200)